Corpus: Europarl v7 (http://www.statmt.org/wmt13/training-parallel-europarl-v7.tgz)
Using:
* Python 3.8.3
* TensorFlow 2.2.0
* SentencePiece 0.1.91

Opening the dataset

In [ ]:
def read_dataset(filename):
    return


Tokenizer

In [ ]:
import sentencepiece as sp

# English

# Training the model for English
sp.SentencePieceTrainer.train(
    input='en_sp_train_data.txt',
    model_prefix='en',
    vocab_size=1000,
)
# Getting the tokenizer object
en_sp = sp.SentencePieceProcessor()
# Loading the English model
en_sp.Load("en.model")

# French

# Training the model for French
sp.SentencePieceTrainer.train(
    input='fr_sp_train_data.txt',
    model_prefix='fr',
    vocab_size=1000,
)
# Getting the tokenizer object
fr_sp = sp.SentencePieceProcessor()
# Loading the French model
fr_sp.Load("fr.model")

In [ ]:
# Testing the English tokenizer
en_test_sentence = "I like green apples."
# Encoding pieces
print(en_sp.EncodeAsPieces(en_test_sentence))
# Encoding pieces as IDs
print(en_sp.EncodeAsIds(en_test_sentence))
# Decoding encoded IDs
print(en_sp.DecodeIds(en_sp.EncodeAsIds(en_test_sentence)))

# Testing the French tokenizer
fr_test_sentence = "J'aime les pommes vertes."
# Encoding pieces
print(fr_sp.EncodeAsPieces(fr_test_sentence))
# Encoding pieces as IDs
print(fr_sp.EncodeAsIds(fr_test_sentence))
# Decoding encoded IDs
print(fr_sp.DecodeIds(fr_sp.EncodeAsIds(fr_test_sentence)))

# Test a mix to see the mess?